<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install import-ipynb
!pip install sentencepiece

In [1]:
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import torch
import import_ipynb
import numpy as np
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
from extractive_summarization import *
%cd ..

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from extractive_summarization.ipynb
/content/drive/MyDrive


In [3]:
df = data_setup(n = 10000) 

In [4]:
df['text_seq'] = df.sentences.map(lambda row: ' '.join(row))
df['text_seq_cleaned'] = df.sentences_cleaned.map(lambda row: ' '.join(row))

In [5]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [25]:
inputs = tokenizer.encode("summarize: " + df.text_seq.head(1).values[0], return_tensors="pt", max_length=1017, truncation=True) # note this is max length, cannot go beyond

Token indices sequence length is longer than the specified maximum sequence length for this model (1017 > 512). Running this sequence through the model will result in indexing errors


In [20]:
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True) # TODO tune!
#https://www.thepythoncode.com/article/text-summarization-using-huggingface-transformers-python
#https://huggingface.co/blog/how-to-generate

In [21]:
print(tokenizer.decode(outputs[0]))

<pad> football clubs around the world are starting to make efforts to become more ecofriendly a 2008 survey by ethical consumer found that manchester city were the greenest among its achievements. german club nuremberg has a cistern that can hold 1000 cubic meters of water and solar panels are used to heat water for the toilets dartford fc have a scheme to provide the city of trondheim with hot water.</s>


In [9]:
print(tokenizer.decode(outputs[0]))

<pad> artificial pitches can save 20000 liters of water a day football stadiums have a huge carbon footprint just keeping a pitch in top condition requires regular watering underpitch heating to prevent freezing in the winter. manchester city have reduced landfill by 85 percent moved to electric vehicles at the ground and used ecofriendly paper for matchday programs. english nonleague club dartford fc has shown that small clubs arent the only ones that can afford to be ecofriendly.</s>


In [14]:
print(tokenizer.decode(outputs[0]))

<pad> football clubs around the world are starting to make efforts to become more eco-friendly. a survey by "ethical consumer" found that manchester city was the greenest. dartford's princes park stadium has a "living roof" of plants that provide a natural air filtration system.</s>


In [ ]:
#summarizer = pipeline("summarization", framework = 'pt')
#summarized = summarizer(article, min_length=0, max_length=100) # max length is number of words/tokens
# max number of tokens can consider: 1024. Need to limit to first 1024
  # ok because best info tends to be at front of articles (and bcg documents bc pyramid principle)
  # potential extension is to fine tune with a higher max allowed

In [ ]:
# https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/